In [1]:
!pip install seaborn catboost --quiet
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn
from keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler,OneHotEncoder
from sklearn.metrics import confusion_matrix
from tensorflow.keras.utils import to_categorical


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


2025-08-11 05:44:23.146499: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import os
import pandas as pd
import kagglehub

# Download the 2018 CICIDS dataset
path_2018 = kagglehub.dataset_download("solarmainframe/ids-intrusion-csv")
print("Dataset folder for 2018:", path_2018)

# List all CSV files in the 2018 dataset directory
csv_files_2018 = [f for f in os.listdir(path_2018) if f.endswith('.csv')]
print(f"Found {len(csv_files_2018)} CSV files in 2018 dataset.")

# Load only first 5 CSV files for faster processing
csv_files_2018_subset = csv_files_2018[:5]

dfs_2018 = []
for file_name in csv_files_2018_subset:
    file_path = os.path.join(path_2018, file_name)
    print("Loading:", file_path)
    df = pd.read_csv(file_path, low_memory=False)
    dfs_2018.append(df)

# Combine loaded CSV files into one DataFrame
df = pd.concat(dfs_2018, ignore_index=True)
print("Shape of combined 2018 data:", df.shape)

# Preview the combined data
df.head()


/Users/macbookpro/Documents/Network-Intrusion-Detection-System/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 1.60G/1.60G [08:51<00:00, 3.23MB/s]

Extracting files...


Dataset folder for 2018: /Users/macbookpro/.cache/kagglehub/datasets/solarmainframe/ids-intrusion-csv/versions/1
Found 10 CSV files in 2018 dataset.
Loading: /Users/macbookpro/.cache/kagglehub/datasets/solarmainframe/ids-intrusion-csv/versions/1/02-14-2018.csv
Loading: /Users/macbookpro/.cache/kagglehub/datasets/solarmainframe/ids-intrusion-csv/versions/1/02-15-2018.csv
Loading: /Users/macbookpro/.cache/kagglehub/datasets/solarmainframe/ids-intrusion-csv/versions/1/02-20-2018.csv
Loading: /Users/macbookpro/.cache/kagglehub/datasets/solarmainframe/ids-intrusion-csv/versions/1/02-21-2018.csv
Loading: /Users/macbookpro/.cache/kagglehub/datasets/solarmainframe/ids-intrusion-csv/versions/1/03-01-2018.csv
Shape of combined 2018 data: (11425598, 84)


,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Flow ID,Src IP,Src Port,Dst IP
0,0,0,14/02/2018 08:31:01,112641719,3,0,0,0,0,0,...,0,56320859.5,139.300036,56320958,56320761,Benign,NaN,NaN,NaN,NaN
1,0,0,14/02/2018 08:33:50,112641466,3,0,0,0,0,0,...,0,56320733.0,114.551299,56320814,56320652,Benign,NaN,NaN,NaN,NaN
2,0,0,14/02/2018 08:36:39,112638623,3,0,0,0,0,0,...,0,56319311.5,301.934596,56319525,56319098,Benign,NaN,NaN,NaN,NaN
3,22,6,14/02/2018 08:40:13,6453966,15,10,1239,2273,744,0,...,0,0.0,0.0,0,0,Benign,NaN,NaN,NaN,NaN
4,22,6,14/02/2018 08:40:23,8804066,14,11,1143,2209,744,0,...,0,0.0,0.0,0,0,Benign,NaN,NaN,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11425598 entries, 0 to 11425597
Data columns (total 84 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Dst Port           object 
 1   Protocol           object 
 2   Timestamp          object 
 3   Flow Duration      object 
 4   Tot Fwd Pkts       object 
 5   Tot Bwd Pkts       object 
 6   TotLen Fwd Pkts    object 
 7   TotLen Bwd Pkts    object 
 8   Fwd Pkt Len Max    object 
 9   Fwd Pkt Len Min    object 
 10  Fwd Pkt Len Mean   object 
 11  Fwd Pkt Len Std    object 
 12  Bwd Pkt Len Max    object 
 13  Bwd Pkt Len Min    object 
 14  Bwd Pkt Len Mean   object 
 15  Bwd Pkt Len Std    object 
 16  Flow Byts/s        object 
 17  Flow Pkts/s        object 
 18  Flow IAT Mean      object 
 19  Flow IAT Std       object 
 20  Flow IAT Max       object 
 21  Flow IAT Min       object 
 22  Fwd IAT Tot        object 
 23  Fwd IAT Mean       object 
 24  Fwd IAT Std        object 
 25  Fwd IAT Max     

In [4]:
df.columns

Index(['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts',
       'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
      

In [5]:
df['Label'].unique()

array(['Benign', 'FTP-BruteForce', 'SSH-Bruteforce',
       'DoS attacks-GoldenEye', 'DoS attacks-Slowloris',
       'DDoS attacks-LOIC-HTTP', 'DDOS attack-LOIC-UDP',
       'DDOS attack-HOIC', 'Label', 'Infilteration'], dtype=object)

In [6]:
df['Label'].value_counts()

Label
Benign                    9635130
DDOS attack-HOIC           686012
DDoS attacks-LOIC-HTTP     576191
FTP-BruteForce             193360
SSH-Bruteforce             187589
Infilteration               93063
DoS attacks-GoldenEye       41508
DoS attacks-Slowloris       10990
DDOS attack-LOIC-UDP         1730
Label                          25
Name: count, dtype: int64

In [7]:
Label = pd.DataFrame(df.Label)
Label

,Label
0,Benign
1,Benign
2,Benign
3,Benign
4,Benign
...,...
11425593,Label
11425594,Label
11425595,Label
11425596,Label


In [8]:
from sklearn.preprocessing import MinMaxScaler

std_scaler = MinMaxScaler()
def standardization(df,col):
    for i in col:
        arr = df[i]
        arr = np.array(arr)
        df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
    return df

numeric_col = df.select_dtypes(include='number').columns
df = standardization(df,numeric_col)

In [10]:
X = df.drop('Label', axis=1)
y = df['Label']

In [11]:
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1))
y_encoded = pd.DataFrame(y_encoded)
y_encoded

,0,1,2,3,4,5,6,7,8,9
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
11425593,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
11425594,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
11425595,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
11425596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
df = pd.concat([X, y_encoded], axis=1)

In [13]:
X_train,X_test,y_train,y_test=train_test_split(X,y_encoded,test_size=0.2)
print(X_train.shape," ",X_test.shape)
print(y_train.shape," ",y_test.shape)

(9140478, 83)   (2285120, 83)
(9140478, 10)   (2285120, 10)


In [14]:
#selecting 1% of random rows for better running time

X_train = X_train.sample(frac=0.1, replace=True, random_state=1)
y_train = y_train.sample(frac=0.1, replace=True, random_state=1)
X_test = X_test.sample(frac=0.1, replace=True, random_state=1)
y_test = y_test.sample(frac=0.1, replace=True, random_state=1)
print (X_train.shape, y_train.shape)
print( X_test.shape, y_test.shape)

(914048, 83) (914048, 10)
(228512, 83) (228512, 10)


In [17]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd

# Identify categorical columns in X_train
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns

# One-hot encode categorical features for train
X_train_encoded = pd.get_dummies(X_train, columns=cat_cols)

# One-hot encode categorical features for test (to keep same columns)
X_test_encoded = pd.get_dummies(X_test, columns=cat_cols)

# Align columns of train and test dataframes (fill missing columns with 0)
X_train_encoded, X_test_encoded = X_train_encoded.align(X_test_encoded, join='left', axis=1, fill_value=0)

# Now create and fit SelectFromModel
sel = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1))
sel.fit(X_train_encoded, y_train)

# Get selected features (names)
features = X_train_encoded.columns[sel.get_support()]
print(features)
print(len(features))

# Mean feature importance
print(np.mean(sel.estimator_.feature_importances_))
print(sel.estimator_.feature_importances_)

# Transform train and test sets
X_train_rfe = sel.transform(X_train_encoded)
X_test_rfe = sel.transform(X_test_encoded)

: 

In [ ]:
X_train_rfe=np.array(X_train_rfe)
X_test_rfe=np.array(X_test_rfe)
y=np.array(y)

In [ ]:
X_train = np.reshape(X_train_rfe, ( X_train_rfe.shape[0], 1 , X_train_rfe.shape[1] ))
X_test = np.reshape(X_test_rfe, ( X_test_rfe.shape[0], 1,  X_test_rfe.shape[1] ))

In [ ]:
from keras.layers import Dense, LSTM, MaxPool1D, Flatten, Dropout # importing dense layer
from keras.models import Sequential #importing Sequential layer
from keras.layers import Input
from keras.models import Model

model = Sequential() # initializing model
model.add(LSTM(64,return_sequences=True,input_shape = (1, X_train_rfe.shape[1])))
model.add(Dropout(0.2))
model.add(LSTM(64,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64,return_sequences=True))
model.add(Flatten())
model.add(Dense(units=50))
# output layer with softmax activation
model.add(Dense(units=7,activation='softmax'))

In [ ]:
# defining loss function, optimizer, metrics and then compiling model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# summary of model layers
model.summary()

In [ ]:
!nvidia-smi # ONLY IN COLAB, if you are using Macbook, you need to change it according to your run time

In [ ]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')

In [ ]:
import tensorflow as tf
import subprocess

class GpuUsageLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"\n--- Epoch {epoch+1} GPU Usage ---")
        print(subprocess.getoutput("nvidia-smi --query-gpu=memory.used,memory.total,utilization.gpu --format=csv"))

with tf.device('/GPU:0'):
    history = model.fit(
        X_train, y_train,
        epochs=100,  # use small number for test
        batch_size=5000,
        validation_split=0.2,
        callbacks=[GpuUsageLogger()]
    )


In [ ]:
# predicting target attribute on testing dataset
test_results = model.evaluate(X_test, y_test, verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')

In [ ]:
model.save('2018_LSTM_feature_extraction.keras')